In this notebook, we show how we can train a model with Tensorflow and save it as a TileDB array on TileDB-Cloud.
Firstly, let's import what we need.

In [1]:
import os

import numpy as np
import tensorflow as tf
import tiledb.cloud

from tiledb.ml.models.tensorflow_keras import TensorflowKerasTileDBModel

We then create a TileDB-Cloud context, log in to TileDB via our API token (or username/password) and get our username.

In [2]:
ctx = tiledb.cloud.Ctx()
tiledb.cloud.login(token=os.getenv('TILEDB_API_TOKEN'))
namespace = tiledb.cloud.client.default_user().username

And move on with training a Tensorflow model with some random data.

In [3]:
X_train = np.random.random((1000, 784))
y_train = np.random.randint(9, size=1000)

X_test = np.random.random((500, 784))
y_test = np.random.randint(9, size=500)

# Create a model
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


print("Model fit...")
model.fit(X_train, y_train, epochs=2)

Model fit...
Epoch 1/2


2022-08-10 21:30:54.521562: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


32/32 [==============================] - 0s 1ms/step - loss: 2.3447 - accuracy: 0.1100
Epoch 2/2
32/32 [==============================] - 0s 1ms/step - loss: 2.2094 - accuracy: 0.1440


We can move on by defining a TileDBTensorflow model and use model save functionality in order to save it directly to
our bucket on S3 (defined with AWS credentials in your TileDB-Cloud account) and register it on TileDB-Cloud.

In [4]:
print('Defining TensorflowKerasTileDBModel model...')
# In order to save our model on S3 and register it on TileDB-Cloud we have to pass our Namespace and TileDB Context.
tiledb_model = TensorflowKerasTileDBModel(uri='tiledb-tensorflow-model', namespace=namespace, ctx=ctx, model=model)

# We will need the uri that was created from our model class
# (and follows pattern tiledb://my_username/s3://my_bucket/my_array),
# in order to interact with our model on TileDB-Cloud.
tiledb_cloud_model_uri = tiledb_model.uri

print('Saving model on S3 and registering on TileDB-Cloud...')
tiledb_model.save()

Defining TensorflowKerasTileDBModel model...
Saving model on S3 and registering on TileDB-Cloud...


Finally, we can use TileDB-Cloud API as described in our [cloud documentation](https://docs.tiledb.com/cloud/), in order
to list our models, get information and deregister them.

In [5]:
# List all our models. All machine learning model TileDB arrays are of type 'ml_model'
print(tiledb.cloud.client.list_arrays(file_type=['ml_model'], namespace=namespace))

# Get model's info
print(tiledb.cloud.array.info(tiledb_cloud_model_uri))

# Load our model for inference
loaded_model = TensorflowKerasTileDBModel(uri=os.path.basename(tiledb_cloud_model_uri), namespace=namespace, ctx=ctx).load()

np.testing.assert_array_equal(model.predict(X_test), loaded_model.predict(X_test))

{'arrays': [{'access_credentials_name': 'gsk',
             'allowed_actions': ['read',
                                 'read_array_logs',
                                 'read_array_info',
                                 'read_array_schema',
                                 'write',
                                 'edit'],
             'description': None,
             'file_properties': None,
             'file_type': 'ml_model',
             'id': 'e5c3483e-215b-44da-af28-b1e385f0cc63',
             'is_favorite': None,
             'last_accessed': datetime.datetime(2022, 8, 10, 18, 31, 18, tzinfo=tzutc()),
             'license_id': None,
             'license_text': None,
             'logo': None,
             'name': 'tiledb-tensorflow-model',
             'namespace': 'george.sakkis',
             'namespace_subscribed': None,
             'pricing': None,
             'public_share': False,
             'read_only': False,
             'share_count': None,
             's

In [6]:
# Deregister model and physically delete the array from the S3 bucket
tiledb.cloud.array.delete_array(tiledb_cloud_model_uri)